In [1]:
library(RPostgreSQL)
library(tidyverse)

Loading required package: DBI
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.4.2     ✔ dplyr   0.7.4
✔ tidyr   0.8.0     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
data_dir <- file.path("..", "data")
sql_dir <- file.path("..", "sql")

In [3]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [4]:
sql <- readr::read_file(file.path(sql_dir, "tte-summary.sql"))
sql

[1] "with tte_1 as (\n    select icu.icustay_id, ne.chartdate as echo_time\n    from icustays icu left join noteevents ne using (hadm_id)\n    where lower(ne.category) like 'echo'\n    and lower(ne.description) like 'report'\n    and (ne.chartdate between (date_trunc('day', icu.intime)) and icu.outtime)\n)\n\n, tte_2 as (\n    select icustay_id, count(*) as echo_times\n    from tte_1\n    group by icustay_id\n)\n\nselect icustay_id, echo_times\nfrom merged_data left join tte_2 using (icustay_id) where echo = 1;\n"

In [5]:
tte_summary <- dbGetQuery(con, sql)
head(tte_summary)
str(tte_summary)

icustay_id,echo_times
228416,1
274174,1
214954,1
264413,1
214106,1
294908,1


'data.frame':	3063 obs. of  2 variables:
 $ icustay_id: int  228416 274174 214954 264413 214106 294908 232091 222088 213342 213317 ...
 $ echo_times: num  1 1 1 1 1 1 1 1 1 1 ...


In [6]:
tte_summary %>%
summarise(mean = mean(echo_times),
          sd = sd(echo_times))

mean,sd
1.263467,0.6853849


In [7]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE

In [8]:
data.table::fwrite(tte_summary, file.path(data_dir, "echo_times.csv"))